In [1]:
# dependencies
# reference: fourth week's third day's first activity
# xpert
import pandas as pd
import geopandas as gpd
# instructor's resources on slack
    # https://realpython.com/python-folium-web-maps-from-data/#install-folium
import folium
# https://python-visualization.github.io/folium/latest/advanced_guide/colormaps.html
# tutoring session
import branca.colormap as cm
# https://stackoverflow.com/questions/49699048/numpy-pandas-remove-outliers
import numpy as np

In [2]:
# dependency
# fourth week's first day's ninth activity
from pathlib import Path

In [3]:
# path
# fourth week's first day's ninth activity
# tutoring session
PATHtochoropleth = Path("Data/choropleth.geojson")

In [4]:
# reading choropleth.geojson
# tutoring session
normalplusoutliers = gpd.read_file(PATHtochoropleth)
# verification
# tutoring session
print(normalplusoutliers.shape)
normalplusoutliers.head()

(2672, 4)


,county,state,counts,geometry
0,los angeles,california,1736,"MULTIPOLYGON (((-118.67822 33.03973, -118.6785..."
1,maricopa,arizona,1274,"POLYGON ((-111.89310 33.20471, -111.89310 33.2..."
2,king,washington,1095,"POLYGON ((-122.00036 47.77540, -122.00038 47.7..."
3,san diego,california,808,"POLYGON ((-116.85262 32.55747, -116.85088 32.5..."
4,cook,illinois,781,"POLYGON ((-87.52500 41.64464, -87.52485 41.647..."


In [6]:
# getting higher, less common values; getting normal values
# https://stackoverflow.com/questions/49699048/numpy-pandas-remove-outliers
def yesoutliers(data):
    q1, q3 = np.percentile(data, [25, 75])
    iqr = q3 - q1
    lowerbound = q1 - (iqr * 1.5)
    upperbound = q3 + (iqr * 1.5)
    # xpert
    return np.where(data > upperbound)

In [7]:
# outliers, yes outliers
# https://stackoverflow.com/questions/49699048/numpy-pandas-remove-outliers
# xpert
outliers = normalplusoutliers.iloc[yesoutliers(normalplusoutliers["counts"])]
print(outliers.shape)
outliers.head()

(351, 4)


,county,state,counts,geometry
0,los angeles,california,1736,"MULTIPOLYGON (((-118.67822 33.03973, -118.6785..."
1,maricopa,arizona,1274,"POLYGON ((-111.89310 33.20471, -111.89310 33.2..."
2,king,washington,1095,"POLYGON ((-122.00036 47.77540, -122.00038 47.7..."
3,san diego,california,808,"POLYGON ((-116.85262 32.55747, -116.85088 32.5..."
4,cook,illinois,781,"POLYGON ((-87.52500 41.64464, -87.52485 41.647..."


In [8]:
# base
# https://python-visualization.github.io/folium/latest/user_guide/geojson/choropleth.html
# https://www.openstreetmap.org/#map=3/38.01/-95.84
base = folium.Map(location=[38.01, -95.84], zoom_start=3)

In [9]:
# popup
# https://python-visualization.github.io/folium/latest/user_guide/geojson/geojson_popup_and_tooltip.html
popup = folium.GeoJsonPopup(
    fields=["county", "counts"],
    aliases=["county", "sightings"],
    labels=True,
)

In [10]:
# gradient
# https://python-visualization.github.io/folium/latest/advanced_guide/colormaps.html
# tutoring session
gradient = cm.linear.PuBuGn_09.scale(outliers["counts"].min(), outliers["counts"].max())
# https://python-visualization.github.io/folium/latest/advanced_guide/colormaps.html#Draw-a-ColorMap-on-a-map
gradient.caption = "sightings"
gradient

In [11]:
# adding gradient to base
# https://python-visualization.github.io/folium/latest/advanced_guide/colormaps.html#Draw-a-ColorMap-on-a-map
base.add_child(gradient)

In [12]:
# function to determine which color belongs to which county
# tutoring session
def style(feature):
    count = feature["properties"]["counts"]
    colorforcount = gradient(count)
    dict = {
        "fillColor": colorforcount,
        "color": "black",
        "fillOpacity": 1,
        "weight": 0.1,
    }
    return dict

In [14]:
# adding counties and colors to base
# https://python-visualization.github.io/folium/latest/user_guide/geojson/geojson_popup_and_tooltip.html
# https://python-visualization.github.io/folium/latest/user_guide/ui_elements/layer_control.html
# tutoring session
folium.GeoJson(outliers, style_function=style, popup=popup,).add_to(base)

In [15]:
# saving base
# xpert
base.save("yesoutliers.html")